In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


In [ ]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

In [ ]:
pulp.pulpTestAll()

# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [ ]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

In [ ]:
#x1=chicken x2=beef x3=sardines
from pulp import *

In [ ]:
prob = LpProblem("The Whiskas Problem",LpMinimize) #zmienna prob, funckja LpProblem. Mozemy wybrac minimalizacje lub maksymalizacje zadania.
x1=LpVariable("ChickenPercent",0,100) #dodawanie zmiennej decyzyjnej. Drugi i trzeci parametr to są widelki. Czwarty parametr decyduje czy jest to dyskretna wartość (np. liczba puszek), czy ciągła. Defaultowo ciagla.
x2=LpVariable("BeefPercent",0,100) # w tym wypadku nie mozemy użyć mniej niż 0 składnika
x3=LpVariable("SardinePrecent",0,100)

#teraz warunki / PROBLEM DATA - np. ile musimy dodać sardynki, żeby otrzymać minimum białka
#operator +=

#najpierw funkcja kosztu na produkt (per 100 gram) OBJECTIVE FUNCTION - bo obiektem jest optymalna cena za puche
prob += 0.013*x1 + 0.008*x2 + 0.007*x3, "Value per Can"

prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.08*x2 + 0.04*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.09*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0*x3<= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0*x3 <= 0.4, "SaltRequirement"

#tutaj dodamy ograniczenie co do witaminy, w tabelce są mikrogramy
prob += 0.3*0.00000001*x1 + 2.6*0.00000001*x2 + 8.9*0.00000001*x3 >= 0.8*0.000001, "VitaminRequirement"

#zapis do pliku
prob.writeLP("WhiskasModel.lp")

/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


[BeefPercent, ChickenPercent, SardinePrecent]

In [ ]:
#sprawdzamy wartosc
!cat "WhiskasModel.lp"

#wykonujemy optymalizacje
prob.solve()

#sprawdzamy status operacji
print("Status:", LpStatus[prob.status]) 

#sprawdzamy wynik
for v in prob.variables():
    print(v.name, "=", v.varValue)

#wyswietlamy koszt
print("Total Cost of Ingredients per can = ", value(prob.objective))

\* The_Whiskas_Problem *\
Minimize
Value_per_Can: 0.008 BeefPercent + 0.013 ChickenPercent + 0.007 SardinePrecent
Subject To
FatRequirement: 0.1 BeefPercent + 0.08 ChickenPercent + 0.09 SardinePrecent
 >= 6
FibreRequirement: 0.005 BeefPercent + 0.001 ChickenPercent <= 2
PercentagesSum: BeefPercent + ChickenPercent + SardinePrecent = 100
ProteinRequirement: 0.08 BeefPercent + 0.1 ChickenPercent
 + 0.04 SardinePrecent >= 8
SaltRequirement: 0.005 BeefPercent + 0.002 ChickenPercent <= 0.4
VitaminRequirement: 2.6e-08 BeefPercent + 3e-09 ChickenPercent
 + 8.9e-08 SardinePrecent >= 8e-07
Bounds
BeefPercent <= 100
ChickenPercent <= 100
SardinePrecent <= 100
End
Status: Optimal
BeefPercent = 72.727273
ChickenPercent = 18.181818
SardinePrecent = 9.0909091
Total Cost of Ingredients per can =  0.8818181817
